# Testing

The developed classes need some thorough testing before we run the whole experiment. It would be very bad if the script breaks in the middle of tuning. This notebook lists some situations that should be tested and explains how to do so.

The tests can be performed by evaluating the objective function for a specified set of parameter values. Every class has a method <i>objfunction</i>, which takes a list of values as input.

In [2]:
import os
os.chdir('..')
import helpers
import models
from models import OneShotCNN, OneShotTransferCNN

Using TensorFlow backend.


In [3]:
# initialize with logging and verbose
model = OneShotCNN(log=True, verbose=1)
# choose lower bound of model as example (THIS SHOULD BE DONE MANUALLY TO PERFORM THE TESTS)
x = model.xlow
print(x)

[  1.00000000e-04   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.00000000e+00
   0.00000000e+00   0.00000000e+00   3.00000000e+00   4.00000000e+00
   5.00000000e+00   0.00000000e+00   0.00000000e+00   5.00000000e+01]


As you can see, x is just a list of numbers. This can be put in the objective function

In [4]:
# test for given x
model.objfunction(x)

-------------
Experiment 1.
-------------
learning_rate: 0.0001
dropout_rate1: 0.0
dropout_rate2: 0.0
width_shift: 0.0
height_shift: 0.0
shear: 0.0
zoom: 0.0
num_conv_layers: 2.0
num_dense_layers: 0.0
num_maxpools: 0.0
neurons_first_conv: 3.0
neurons_remaining_conv: 4.0
neurons_dense: 5.0
rotation: 0.0
horizontal_flip: 0.0
epochs: 50.0
-------------
Cross validating..
fit 1:
test accuracy: 46.76%.
fit 2:
test accuracy: 64.38%.
fit 3:
test accuracy: 72.9%.
fit 4:
test accuracy: 50.26%.
fit 5:
test accuracy: 68.56%.
fit 6:
test accuracy: 47.32%.
fit 7:
test accuracy: 60.44%.
fit 8:
test accuracy: 45.92%.
fit 9:
test accuracy: 52.04%.
fit 10:
test accuracy: 61.28%.
Scores: [0.46759998798370361, 0.64380002021789551, 0.72899997234344482, 0.50260001420974731, 0.68559998273849487, 0.4731999933719635, 0.60439997911453247, 0.45919999480247498, 0.52039998769760132, 0.6128000020980835].
Mean: 56.99%. Standard deviation: 9.29%


-0.56985999345779415

The <i>hyper_map</i> attribute of the model can be used to get the index of a hyperparameter in X. For example, the learning rate is the first entry of X (index 0).

In [5]:
model.hyper_map['learning_rate']

0

to see the full list of available hyperparameters (and their order), you can print the <i>hyperparams</i> attribute:

In [7]:
model.hyperparams

['learning_rate',
 'dropout_rate1',
 'dropout_rate2',
 'width_shift',
 'height_shift',
 'shear',
 'zoom',
 'num_conv_layers',
 'num_dense_layers',
 'num_maxpools',
 'neurons_first_conv',
 'neurons_remaining_conv',
 'neurons_dense',
 'rotation',
 'horizontal_flip',
 'epochs']

# OneShotCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.

# OneShotTransferCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.
- Having 'num_fixed_layers' larger than the total number of layers in the model.
- ..